In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/s5e11-dataset2/submission (4).csv
/kaggle/input/s5e11-dataset2/submission (5).csv
/kaggle/input/playground-series-s5e11/sample_submission.csv
/kaggle/input/playground-series-s5e11/train.csv
/kaggle/input/playground-series-s5e11/test.csv


In [2]:
# Define the blending function
def blend_submissions(weight_dict, output_path):
    # Initialize list to store loaded DataFrames
    dataframes = []

    # Load each submission with its weight
    for path, weight in weight_dict.items():
        # Read the CSV file
        df = pd.read_csv(path)

        # Add a weighted prediction column
        df["weighted_pred"] = df["loan_paid_back"] * weight

        # Append to list
        dataframes.append(df[["id", "weighted_pred"]])

    # Merge all submissions on 'id'
    merged = dataframes[0]
    for df in dataframes[1:]:
        # Merge on id
        merged = merged.merge(df, on="id", how="inner", suffixes=("", "_dup"))

        # Combine duplicate weighted_pred columns if any
        if "weighted_pred_dup" in merged.columns:
            merged["weighted_pred"] += merged["weighted_pred_dup"]
            merged.drop(columns=["weighted_pred_dup"], inplace=True)

    # Compute total weight
    total_weight = sum(weight_dict.values())

    # Compute blended prediction
    merged["loan_paid_back"] = merged["weighted_pred"] / total_weight

    # Prepare final DataFrame
    blended = merged[["id", "loan_paid_back"]]

    # Save blended submission
    blended.to_csv(output_path, index=False)

    # Print confirmation
    print(f"✅ Blended submission saved to {output_path}")

In [3]:
# Define the main function
def main():
    # Define file paths and their respective weights
    weight_dict = {
        "/kaggle/input/s5e11-dataset2/submission (4).csv": 2.34,
        "/kaggle/input/s5e11-dataset2/submission (5).csv": 0.3,
    }

    # Call blend function
    blend_submissions(weight_dict, output_path="submission.csv")

In [4]:
# Call the main function
if __name__ == "__main__":
    main()

✅ Blended submission saved to submission.csv


# PLZ do UPVOTE the NOTEBOOK